<a href="https://colab.research.google.com/github/EwFox76/IMERS-O-ALURA-IA/blob/main/Agende_de_suporte_emocional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar bibliotecas necessárias
!pip install -q google-genai ipywidgets google-adk

# Importações
import os
from google.colab import userdata
from IPython.display import display
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import Textarea, Button, Output, HBox, VBox, Layout

from google import genai
import google.adk as adk
from google.genai import types

# Chave de API
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Cliente e modelo
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Instruções do agente principal
chat_config = types.GenerateContentConfig(system_instruction="""
Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto, dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
É FUNDAMENTAL que você NUNCA aja como um profissional de saúde mental.
Suas respostas devem ser:
- Empáticas e acolhedoras.
- Focadas em positividade e encorajamento.
- Oferecer dicas GERAIS de bem-estar.
- NUNCA fazer diagnósticos ou sugerir tratamentos ou medicamentos.
- SEMPRE incluir um DISCLAIMER claro no início ou final de CADA interação de que você não é um profissional e RECOMENDAR FORTEMENTE a busca por AJUDA PROFISSIONAL.
""")

# Criar o chat principal
chat = client.chats.create(model=model, config=chat_config)

# Criar os agentes auxiliares
chat_frases = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você é um buscador de frases e poesias motivacionais. Quando solicitado, responda com trechos que inspirem força, esperança e coragem.
""")
)

chat_musicas = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você é um recomendador de músicas motivacionais. Quando solicitado, sugira músicas que animem e inspirem alguém que se sinta desanimado.
""")
)

# Variável de controle
chat_active = True

# Widgets
output_area = Output(layout={
    'border': '4px solid black',
    'width': 'auto',
    'height': '300px',
    'overflow_y': 'auto'
})

input_area = Textarea(
    placeholder='Digite seus sentimentos aqui...',
    layout=Layout(width='auto', height='100px')
)

send_button = Button(
    description='Enviar',
    style={'button_color': 'darkgreen', 'font_weight': 'bold'}
)

exit_button = Button(
    description='Sair da Conversa',
    style={'button_color': 'salmon', 'font_weight': 'bold'}
)

# Layout
button_box = HBox([send_button, exit_button], layout=Layout(justify_content='space-between'))

# Funções
def on_send_button_clicked(b):
    global chat_active
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_input:
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return

        with output_area:
            print(f"{datetime.now().strftime('%H:%M')} - Você: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")

            # Verifica se há sinais de tristeza
            palavras_tristeza = ['triste', 'cansado', 'sozinho', 'desanimado', 'angustiado', 'deprimido', 'chateado', 'sem forças']
            if any(p in user_input.lower() for p in palavras_tristeza):
                print("Percebo que você pode estar passando por um momento difícil. Aqui estão algumas inspirações:\n")

                resposta_frases = chat_frases.send_message("Traga uma frase ou poesia motivacional para alguém triste.")
                print(f"💬 Frase motivacional: {resposta_frases.text}\n")

                resposta_musicas = chat_musicas.send_message("Sugira uma música que anime alguém desanimado.")
                print(f"🎵 Música motivacional: {resposta_musicas.text}\n")

def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    exit_button.description = "Saída Solicitada"
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar!")

send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Exibição
print("Bem-vindo! Clique no botão 'Sair da Conversa' a qualquer momento para encerrar.")
display(VBox([output_area, input_area, button_box]))

# Mensagem inicial com disclaimer
with output_area:
    response = chat.send_message("Olá, estou aqui para conversar.")
    print(f"Resposta: {response.text}\n")


In [ ]:
# Instalar bibliotecas (executa apenas uma vez)
!pip install -q google-genai ipywidgets google-adk

# Importações
import os
from google.colab import userdata
from IPython.display import display
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import Textarea, Button, Output, HBox, VBox, Layout

from google import genai
import google.adk as adk
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Agente principal: Apoio emocional
chat_principal = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto, dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
IMPORTANTE: NÃO é profissional de saúde. Sempre inclua um aviso de que é apenas uma IA e recomende ajuda profissional.
""")
)

# Agente 2: Frases/Poesias motivacionais
chat_frases = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você fornece frases e poesias motivacionais para encorajar alguém que se sente mal.
Responda com uma citação bonita e inspiradora. Seja breve e impactante.
""")
)

# Agente 3: Músicas motivacionais
chat_musicas = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você recomenda músicas motivacionais para quem está desanimado, triste ou sem esperança.
Sugira títulos e artistas com tom encorajador.
""")
)

# Variável de controle
chat_ativo = True

# Interface com widgets
output_area = Output(layout={
    'border': '3px solid black',
    'width': 'auto',
    'height': '300px',
    'overflow_y': 'auto',
    'padding': '10px'
})

input_area = Textarea(
    placeholder='Como você está se sentindo hoje?',
    layout=Layout(width='auto', height='100px')
)

send_button = Button(
    description='Enviar',
    style={'button_color': 'darkgreen', 'font_weight': 'bold'}
)

exit_button = Button(
    description='Sair da Conversa',
    style={'button_color': 'salmon', 'font_weight': 'bold'}
)

button_box = HBox([send_button, exit_button], layout=Layout(justify_content='space-between'))

# Função de envio
def on_send_button_clicked(b):
    global chat_ativo
    if not chat_ativo:
        return

    user_input = input_area.value.strip()
    input_area.value = ''

# Instalar bibliotecas (executa apenas uma vez)
!pip install -q google-genai ipywidgets google-adk

# Importações
import os
from google.colab import userdata
from IPython.display import display
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import Textarea, Button, Output, HBox, VBox, Layout

from google import genai
import google.adk as adk
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Agente principal: Apoio emocional
chat_principal = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto, dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
IMPORTANTE: NÃO é profissional de saúde. Sempre inclua um aviso de que é apenas uma IA e recomende ajuda profissional.
""")
)

# Agente 2: Frases/Poesias motivacionais
chat_frases = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você fornece frases e poesias motivacionais para encorajar alguém que se sente mal.
Responda com uma citação bonita e inspiradora. Seja breve e impactante.
""")
)

# Agente 3: Músicas motivacionais
chat_musicas = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você recomenda músicas motivacionais para quem está desanimado, triste ou sem esperança.
Sugira títulos e artistas com tom encorajador.
""")
)

# Variável de controle
chat_ativo = True

# Interface com widgets
output_area = Output(layout={
    'border': '3px solid black',
    'width': 'auto',
    'height': '300px',
    'overflow_y': 'auto',
    'padding': '10px'
})

input_area = Textarea(
    placeholder='Como você está se sentindo hoje?',
    layout=Layout(width='auto', height='100px')
)

send_button = Button(
    description='Enviar',
    style={'button_color': 'darkgreen', 'font_weight': 'bold'}
)

exit_button = Button(
    description='Sair da Conversa',
    style={'button_color': 'salmon', 'font_weight': 'bold'}
)

button_box = HBox([send_button, exit_button], layout=Layout(justify_content='space-between'))

# Função de envio
def on_send_button_clicked(b):
    global chat_ativo
    if not chat_ativo:
        return

    user_input = input_area.value.strip()
    input_area.value = ''

    if not user_input:
        return
    if user_input.lower() == 'sair':
        with output_area:
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Você: sair")
            print(f"[{datetime.now().strftime('%H:%M:%S')}] IA: Espero ter ajudado. Lembre-se de buscar ajuda profissional se precisar.")
        chat_ativo = False
        send_button.disabled = True
        input_area.disabled = True
        return

    with output_area:
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Você: {user_input}")

    # Determinar qual agente usar com base na entrada
    if any(keyword in user_input.lower() for keyword in ['frase', 'poesia', 'citação', 'motivacional']):
        response = chat_frases.send_message(user_input)
    elif any(keyword in user_input.lower() for keyword in ['música', 'cantar', 'ouvir']):
        response = chat_musicas.send_message(user_input)
    else:
        response = chat_principal.send_message(user_input)

    with output_area:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] IA: {response.text}")

# Função para sair
def on_exit_button_clicked(b):
    global chat_ativo
    if chat_ativo:
        with output_area:
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] Você: sair")
            print(f"[{datetime.now().strftime('%H:%M:%S')}] IA: Espero ter ajudado. Lembre-se de buscar ajuda profissional se precisar.")
        chat_ativo = False
        send_button.disabled = True
        input_area.disabled = True

# Conectar botões às funções
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Display da interface
chat_interface = VBox([output_area, input_area, button_box])
display(chat_interface)

# Mensagem inicial
with output_area:
    print(f"[{datetime.now().strftime('%H:%M:%S')}] IA: Olá! Como você está se sentindo hoje? Estou aqui para ouvir e oferecer um pouco de apoio. Lembre-se que sou apenas uma inteligência artificial e não substituo ajuda profissional.")

In [ ]:
# Instalar bibliotecas (execute uma vez)
!pip install -q google-genai ipywidgets google-adk duckduckgo-search

# Importações
import os
from google.colab import userdata
from IPython.display import display, HTML
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import Textarea, Button, Output, HBox, VBox, Layout

from google import genai
import google.adk as adk
from google.genai import types

# 🔍 Buscas
from duckduckgo_search import DDGS

def buscar_link(query):
    with DDGS() as ddgs:
        resultados = ddgs.text(query, region='wt-wt', safesearch='moderate', max_results=1)
        for r in resultados:
            return r['href']
    return None

# Chave da API
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Agente principal: Apoio emocional
chat_principal = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você é um agente de suporte positivo e encorajador. Ofereça conforto emocional, sem diagnóstico. Sempre inclua o aviso de que é apenas uma IA e recomende ajuda profissional.
""")
)

# Agente 2: Frases/Poesias motivacionais
chat_frases = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você fornece frases e poesias motivacionais para encorajar alguém que se sente mal. Responda com uma citação ou mini poesia com tom leve e bonito.
""")
)

# Agente 3: Músicas motivacionais
chat_musicas = client.chats.create(
    model=model,
    config=types.GenerateContentConfig(system_instruction="""
Você recomenda músicas motivacionais para quem está triste ou desanimado. Responda com nome da música e artista.
""")
)

# Variável de controle
chat_ativo = True

# Widgets
output_area = Output(layout={
    'border': '3px solid black',
    'width': 'auto',
    'height': '300px',
    'overflow_y': 'auto',
    'padding': '10px'
})

input_area = Textarea(
    placeholder='Como você está se sentindo hoje?',
    layout=Layout(width='auto', height='100px')
)

send_button = Button(
    description='Enviar',
    style={'button_color': 'darkgreen', 'font_weight': 'bold'}
)

exit_button = Button(
    description='Sair da Conversa',
    style={'button_color': 'salmon', 'font_weight': 'bold'}
)

button_box = HBox([send_button, exit_button], layout=Layout(justify_content='space-between'))

# Função principal
def on_send_button_clicked(b):
    global chat_ativo
    if not chat_ativo:
        return

    user_input = input_area.value.strip()
    input_area.value = ''

    if not user_input:
        return
    if user_input.lower() == 'sair':
        on_exit_button_clicked(None)
        return

    with output_area:
        print(f"🧍 Você: {user_input}\n")

        # Resposta principal
        resposta_principal = chat_principal.send_message(user_input)
        print(f"💬 Apoio: {resposta_principal.text}\n")

        # Se estiver em sofrimento...
        palavras_tristeza = ['triste', 'cansado', 'sozinho', 'desanimado', 'angustiado', 'deprimido', 'chateado', 'sem forças']
        if any(p in user_input.lower() for p in palavras_tristeza):
            print("🕊 Parece que você está passando por um momento difícil. Aqui vão algumas inspirações especiais:\n")

            # Frase
            frase = chat_frases.send_message("Uma frase ou poesia motivacional para alguém triste.")
            texto_frase = frase.text.strip()
            link_poesia = buscar_link(f"{texto_frase} site:recantodasletras.com.br") or buscar_link("poesia motivacional site:recantodasletras.com.br")
            if link_poesia:
                display(HTML(f"💖 <b>Frase:</b> {texto_frase}<br>🔗 <a href='{link_poesia}' target='_blank'>Ler mais</a><br><br>"))
            else:
                print(f"💖 Frase: {texto_frase}\n")

            # Música
            musica = chat_musicas.send_message("Sugira uma música motivacional para alguém desanimado.")
            texto_musica = musica.text.strip()
            link_musica = buscar_link(f"{texto_musica} site:youtube.com") or buscar_link(f"{texto_musica} música motivacional")
            if link_musica:
                display(HTML(f"🎵 <b>Música:</b> {texto_musica}<br>🔗 <a href='{link_musica}' target='_blank'>Ouvir no YouTube</a><br><br>"))
            else:
                print(f"🎵 Música: {texto_musica}\n")

# Sair da conversa
def on_exit_button_clicked(b):
    global chat_ativo
    chat_ativo = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    exit_button.description = "Conversa Encerrada"
    with output_area:
        print("🛑 Conversa encerrada. Obrigado por compartilhar! Cuide-se.")

# Conectar botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Exibir interface
print("💬 Bem-vindo! Esta é uma conversa de apoio positivo.\n")
print("Digite como está se sentindo e clique em 'Enviar'.")
print("Se quiser encerrar, digite 'sair' ou clique em 'Sair da Conversa'.")
display(VBox([output_area, input_area, button_box]))

# Mensagem inicial com disclaimer
with output_area:
    intro = chat_principal.send_message("Olá, você pode iniciar a conversa?")
    print(f"🤖 {intro.text}\n")


In [ ]:
# Instalar bibliotecas
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2

# Importar bibliotecas
import os
import pickle
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto,
    dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
    Use um tom calmo, sereno e acolhedor, com emojis apropriados.
    Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
    Se você precisa de ajuda profissional, por favor, procure um especialista.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Widgets
chat_active = True
user_name = ""
output_area = Output(layout={'border': '4px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Funções auxiliares Gemini para música/poesia
def buscar_musica_serena(sentimento):
    pesquisa = f"música calma, serena e alegre para quem está se sentindo {sentimento} site:youtube.com"
    resposta = client.generate_content(pesquisa)
    return resposta.text if resposta else "Não foi possível encontrar sugestões agora."

def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    resposta = client.generate_content(prompt)
    return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"

# Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    service = conectar_google_calendar()
    now = datetime.utcnow().isoformat() + 'Z'
    eventos = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=5, singleEvents=True, orderBy='startTime').execute()
    itens = eventos.get('items', [])
    if not itens:
        return "Nenhum evento encontrado."
    resposta = "\n📅 Eventos do dia:\n"
    for evento in itens:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        resposta += f"\n🕒 {inicio} - {evento['summary']}"
    return resposta

# Botão Enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_name:
            user_name = user_input
            with output_area:
                print(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje?\n")
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return
        with output_area:
            print(f"{user_name}: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")
            frases = buscar_frases_inspiradoras()
            print(f"✨ Frases para você:\n{frases}\n")
            if any(emocao in user_input.lower() for emocao in ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]):
                sentimento = user_input.lower()
                musica = buscar_musica_serena(sentimento)
                print(f"🎵 Música sugerida:\n{musica}\n")
            if "agenda" in user_input.lower():
                eventos = mostrar_eventos_do_dia()
                print(eventos)

# Botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Interface
display(output_area, input_area, HBox([send_button, exit_button]))
with output_area:
    print("Olá! 😊 Sou um agente virtual para apoio leve e positivo. Qual é o seu nome?")


In [ ]:
# Instalar bibliotecas
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2

# Importar bibliotecas
import os
import pickle
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto,
    dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
    Use um tom calmo, sereno e acolhedor, com emojis apropriados.
    Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
    Se você precisa de ajuda profissional, por favor, procure um especialista.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Widgets
chat_active = True
user_name = ""
output_area = Output(layout={'border': '4px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Funções auxiliares Gemini para música/poesia
def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    resposta = chat.send_message(prompt)
    return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"

def buscar_musica_serena(sentimento):
    pesquisa = f"música calma, serena e alegre para quem está se sentindo {sentimento} site:youtube.com"
    resposta = chat.send_message(pesquisa)
    return resposta.text if resposta else "Não foi possível encontrar sugestões agora."


# Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    service = conectar_google_calendar()
    now = datetime.utcnow().isoformat() + 'Z'
    eventos = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=5, singleEvents=True, orderBy='startTime').execute()
    itens = eventos.get('items', [])
    if not itens:
        return "Nenhum evento encontrado."
    resposta = "\n📅 Eventos do dia:\n"
    for evento in itens:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        resposta += f"\n🕒 {inicio} - {evento['summary']}"
    return resposta

# Botão Enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_name:
            user_name = user_input
            with output_area:
                print(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje?\n")
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return
        with output_area:
            print(f"{user_name}: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")
            frases = buscar_frases_inspiradoras()
            print(f"✨ Frases para você:\n{frases}\n")
            if any(emocao in user_input.lower() for emocao in ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]):
                sentimento = user_input.lower()
                musica = buscar_musica_serena(sentimento)
                print(f"🎵 Música sugerida:\n{musica}\n")
            if "agenda" in user_input.lower():
                eventos = mostrar_eventos_do_dia()
                print(eventos)

# Botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Interface
display(output_area, input_area, HBox([send_button, exit_button]))
with output_area:
    print("Olá! 😊 Sou um agente virtual para apoio leve e positivo. Qual é o seu nome?")


In [ ]:
# Instalar bibliotecas
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2

# Importar bibliotecas
import os
import pickle
import re
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto,
    dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
    Use um tom calmo, sereno e acolhedor, com emojis apropriados.
    Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
    Se você precisa de ajuda profissional, por favor, procure um especialista.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Widgets
chat_active = True
user_name = ""
output_area = Output(layout={'border': '4px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Funções auxiliares Gemini para música/poesia com links do YouTube
def buscar_musica_serena(sentimento):
    prompt = (
        f"Por favor, sugira 3 músicas calmas, serenas e alegres no YouTube para quem está se sentindo {sentimento}. "
        "Inclua os títulos e links diretos do YouTube."
    )
    resposta = chat.send_message(prompt)
    texto = resposta.text if resposta else "Não foi possível encontrar sugestões agora. 🌿"

    # Extrair links do YouTube do texto usando regex
    links = re.findall(r'(https?://www\.youtube\.com/watch\?v=[\w-]+)', texto)

    if links:
        resultado = "🎵 Aqui estão algumas músicas para você ouvir e relaxar:\n"
        for i, link in enumerate(links, 1):
            resultado += f"{i}. {link}\n"
        return resultado
    else:
        return texto

def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    resposta = chat.send_message(prompt)
    return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"

# Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    service = conectar_google_calendar()
    now = datetime.utcnow().isoformat() + 'Z'
    eventos = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=5, singleEvents=True, orderBy='startTime').execute()
    itens = eventos.get('items', [])
    if not itens:
        return "Nenhum evento encontrado para hoje. 📅"
    resposta = "📅 Seus próximos eventos hoje:\n"
    for evento in itens:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        resumo = evento.get('summary', 'Sem título')
        resposta += f"🕒 {inicio} - {resumo}\n"
    return resposta

# Botão Enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''

        if not user_name:
            user_name = user_input
            with output_area:
                print(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje? Conte para mim. 😊\n")
            return

        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return

        with output_area:
            print(f"{user_name}: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")

            frases = buscar_frases_inspiradoras()
            print(f"✨ Frases para você:\n{frases}\n")

            estados_emocionais = ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]
            if any(emo in user_input.lower() for emo in estados_emocionais):
                musica = buscar_musica_serena(user_input.lower())
                print(f"{musica}\n")

            if "agenda" in user_input.lower():
                eventos = mostrar_eventos_do_dia()
                print(eventos)

# Botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar! 🌸 Cuide-se bem!")

# Conectar botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Interface
display(output_area, input_area, HBox([send_button, exit_button]))

with output_area:
    print("Olá! 😊 Sou um agente virtual para apoio leve e positivo. Qual é o seu nome? 🌸")


In [ ]:
# Instalar bibliotecas
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2

# Importar bibliotecas
import os
import pickle
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto,
    dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
    Use um tom calmo, sereno e acolhedor, com emojis apropriados.
    Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
    Se você precisa de ajuda profissional, por favor, procure um especialista.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Substituto do YouTube: Links sugeridos de músicas relaxantes
musicas_alternativas = [
    "🎵 Bensound – Relaxing: https://www.bensound.com/royalty-free-music/track/slow-motion",
    "🎵 Free Music Archive – Meditation: https://freemusicarchive.org/genre/Meditation/",
    "🎵 Música ambiente – Internet Archive: https://archive.org/details/ambient_music_collection"
]

# Instalar bibliotecas
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2

# Importar bibliotecas
import os
import pickle
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto,
    dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
    Use um tom calmo, sereno e acolhedor, com emojis apropriados.
    Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
    Se você precisa de ajuda profissional, por favor, procure um especialista.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Substituto do YouTube: Links sugeridos de músicas relaxantes
musicas_alternativas = [
    "🎵 Bensound – Relaxing: https://www.bensound.com/royalty-free-music/track/slow-motion",
    "🎵 Free Music Archive – Meditation: https://freemusicarchive.org/genre/Meditation/",
    "🎵 Música ambiente – Internet Archive: https://archive.org/details/ambient_music_collection"
]

def buscar_musicas_reais(sentimento):
    return (
        f"🔕 Não consegui buscar no YouTube, mas aqui estão sugestões de músicas relaxantes para você:\n" +
        "\n".join(musicas_alternativas)
    )

# Widgets
chat_active = True
user_name = ""
output_area = Output(layout={'border': '4px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Funções auxiliares Gemini para frases

def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    resposta = chat.send_message(prompt)
    return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"

# Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    service = conectar_google_calendar()
    now = datetime.utcnow().isoformat() + 'Z'
    eventos = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=5, singleEvents=True, orderBy='startTime').execute()
    itens = eventos.get('items', [])
    if not itens:
        return "Nenhum evento encontrado."
    resposta = "\n📅 Eventos do dia:\n"
    for evento in itens:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        resposta += f"\n🕒 {inicio} - {evento['summary']}"
    return resposta

# Botão Enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_name:
            user_name = user_input
            with output_area:
                print(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje?\n")
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return
        with output_area:
            print(f"{user_name}: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")
            frases = buscar_frases_inspiradoras()
            print(f"✨ Frases para você:\n{frases}\n")
            if any(emocao in user_input.lower() for emocao in ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]):
                sentimento = user_input.lower()
                musica = buscar_musicas_reais(sentimento)
                print(f"🎵 Músicas sugeridas:\n{musica}\n")
            if "agenda" in user_input.lower():
                eventos = mostrar_eventos_do_dia()
                print(eventos)

# Botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Interface
display(output_area, input_area, HBox([send_button, exit_button]))
# Instalar bibliotecas
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2

# Importar bibliotecas
import os
import pickle
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# API Key
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto,
    dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
    Use um tone calmo, sereno e acolhedor, com emojis apropriados.
    Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
    Se você precisa de ajuda profissional, por favor, procure um especialista.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Substituto do YouTube: Links sugeridos de músicas relaxantes
musicas_alternativas = [
    "🎵 Bensound – Relaxing: https://www.bensound.com/royalty-free-music/track/slow-motion",
    "🎵 Free Music Archive – Meditation: https://freemusicarchive.org/genre/Meditation/",
    "🎵 Música ambiente – Internet Archive: https://archive.org/details/ambient_music_collection"
]

def buscar_musicas_reais(sentimento):
    return (
        f"🔕 Não consegui buscar no YouTube, mas aqui estão sugestões de músicas relaxantes para você:\n" +
        "\n".join(musicas_alternativas)
    )

# Widgets
chat_active = True
user_name = ""
output_area = Output(layout={'border': '4px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Funções auxiliares Gemini para frases

def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    resposta = chat.send_message(prompt)
    return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"

# Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    service = conectar_google_calendar()
    now = datetime.utcnow().isoformat() + 'Z'
    eventos = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=5, singleEvents=True, orderBy='startTime').execute()
    itens = eventos.get('items', [])
    if not itens:
        return "Nenhum evento encontrado."
    resposta = "\n📅 Eventos do dia:\n"
    for evento in itens:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        resposta += f"\n🕒 {inicio} - {evento['summary']}"
    return resposta

# Botão Enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_name:
            user_name = user_input
            with output_area:
                print(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje?\n")
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return
        with output_area:
            print(f"{user_name}: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")
            frases = buscar_frases_inspiradoras()
            print(f"✨ Frases para você:\n{frases}\n")
            if any(emocao in user_input.lower() for emocao in ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]):
                sentimento = user_input.lower()
                musica = buscar_musicas_reais(sentimento)
                print(f"🎵 Músicas sugeridas:\n{musica}\n")
            if "agenda" in user_input.lower():
                eventos = mostrar_eventos_do_dia()
                print(eventos)

# Botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Interface
display(output_area, input_area, HBox([send_button, exit_button]))
with output_area:
    print("Olá! 😊 Sou um agente virtual para apoio leve e positivo. Qual é o seu nome?")
chat_active = True
user_name = ""
output_area = Output(layout={'border': '4px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Funções auxiliares Gemini para frases

def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    resposta = chat.send_message(prompt)
    return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"

# Google Calendar API
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    service = conectar_google_calendar()
    now = datetime.utcnow().isoformat() + 'Z'
    eventos = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=5, singleEvents=True, orderBy='startTime').execute()
    itens = eventos.get('items', [])
    if not itens:
        return "Nenhum evento encontrado."
    resposta = "\n📅 Eventos do dia:\n"
    for evento in itens:
        inicio = evento['start'].get('dateTime', evento['start'].get('date'))
        resposta += f"\n🕒 {inicio} - {evento['summary']}"
    return resposta

# Botão Enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_name:
            user_name = user_input
            with output_area:
                print(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje?\n")
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return
        with output_area:
            print(f"{user_name}: {user_input}\n")
            response = chat.send_message(user_input)
            print(f"Resposta: {response.text}\n")
            frases = buscar_frases_inspiradoras()
            print(f"✨ Frases para você:\n{frases}\n")
            if any(emocao in user_input.lower() for emocao in ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]):
                sentimento = user_input.lower()
                musica = buscar_musicas_reais(sentimento)
                print(f"🎵 Músicas sugeridas:\n{musica}\n")
            if "agenda" in user_input.lower():
                eventos = mostrar_eventos_do_dia()
                print(eventos)

# Botão Sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    with output_area:
        print("Conversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Interface
display(output_area, input_area, HBox([send_button, exit_button]))
with output_area:
    print("Olá! 😊 Sou um agente virtual para apoio leve e positivo. Qual é o seu nome?")


In [ ]:
# Instalar bibliotecas necessárias (execute só uma vez)
!pip install -q google-genai google-api-python-client google-auth google-auth-oauthlib ipywidgets

import os
import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display
from googleapiclient.discovery import build
from google import genai
from google.genai import types

# ---------------- CONFIGURAÇÕES - INSIRA SUAS CHAVES AQUI ----------------

# Chave da API Gemini (Generative Language API)
GEMINI_API_KEY = "SUA_CHAVE_GEMINI_AQUI"

# Chave da API Google Custom Search
GOOGLE_API_KEY = "SUA_CHAVE_GOOGLE_CUSTOM_SEARCH_AQUI"

# ID do mecanismo de busca customizado criado no Google Custom Search
CUSTOM_SEARCH_ENGINE_ID = "SEU_CSE_ID"

# -------------------------------------------------------------------------

# Configurar variável de ambiente para Google Search (necessário para googleapiclient)
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Inicializar cliente Gemini com a chave correta
client = genai.Client(api_key=GEMINI_API_KEY)

# Modelo Gemini a ser usado
model = "gemini-2.5-flash-preview-04-17"

# Configuração do chat com instruções do sistema
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto,
    dicas de bem-estar e positividade para pessoas que possam estar passando por momentos difíceis.
    Use um tom calmo, sereno e acolhedor, com emojis apropriados.
    Não ofereça aconselhamento médico ou psicológico.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Função para buscar frases motivacionais usando Gemini
def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas, calmas e serenas de conforto, inspiração e motivação com emojis."
    resposta = chat.send_message(prompt)
    if resposta and resposta.text:
        return resposta.text
    else:
        return "Vamos respirar fundo e seguir em frente. 🌿"

# Função para buscar sites com músicas relaxantes via Google Custom Search API
def buscar_sites_musicas_relaxantes():
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    query = "músicas relaxantes para meditação site:mp3 download free"
    try:
        res = service.cse().list(q=query, cx=CUSTOM_SEARCH_ENGINE_ID, num=3).execute()
        items = res.get("items", [])
        if not items:
            return "Não encontrei sites com músicas relaxantes no momento."
        links = []
        for item in items:
            title = item.get("title")
            link = item.get("link")
            links.append(f"🎵 {title}: {link}")
        return "\n".join(links)
    except Exception as e:
        return f"Erro ao buscar músicas: {e}"

# Interface widgets
output_area = Output(layout={'border': '2px solid #666', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='80px', width='80%'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Encerrar', button_style='danger')

# Variáveis de controle
chat_ativo = True
usuario_nome = ""

# Função para imprimir no output com scroll automático
def imprimir(texto):
    with output_area:
        print(texto)
        output_area.scroll_to_bottom()

# Função para lidar com clique no botão enviar
def on_send_button_clicked(b):
    global chat_ativo, usuario_nome
    if not chat_ativo:
        return

    user_input = input_area.value.strip()
    input_area.value = ''

    if not usuario_nome:
        usuario_nome = user_input
        imprimir(f"🌸 Olá, {usuario_nome}! Como você está se sentindo hoje? 😊")
        return

    if user_input.lower() == 'sair' or user_input.lower() == 'encerrar':
        on_exit_button_clicked(None)
        return

    imprimir(f"{usuario_nome}: {user_input}")

    # Resposta Gemini para o sentimento
    try:
        resposta = chat.send_message(user_input)
        if resposta and resposta.text:
            imprimir(f"🤖: {resposta.text}")
        else:
            imprimir("🤖: Não consegui responder agora. Tente novamente mais tarde.")
    except Exception as e:
        imprimir(f"🤖: Erro ao processar sua mensagem: {e}")

    # Frases inspiradoras
    frases = buscar_frases_inspiradoras()
    imprimir(f"\n✨ Frases inspiradoras:\n{frases}\n")

    # Se a pessoa mencionar sentimentos negativos, sugerir músicas
    sentimentos_negativos = ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]
    if any(palavra in user_input.lower() for palavra in sentimentos_negativos):
        imprimir("🎵 Sugestões de sites com músicas relaxantes para você:\n")
        musicas = buscar_sites_musicas_relaxantes()
        imprimir(musicas)

# Função para encerrar o chat
def on_exit_button_clicked(b):
    global chat_ativo
    chat_ativo = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    imprimir("\nConversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar eventos dos botões
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Mostrar interface
display(VBox([
    output_area,
    HBox([input_area, send_button, exit_button])
]))

imprimir("Olá! 😊 Sou seu agente de bem-estar virtual. Qual é o seu nome?")


In [ ]:
# Instalar bibliotecas necessárias
!pip install -q google-genai ipywidgets google-auth google-api-python-client google-auth-oauthlib google-auth-httplib2 google-auth-httplib2

# Importar bibliotecas
import os
import pickle
from datetime import datetime

from google.colab import userdata
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

import ipywidgets as widgets
from ipywidgets import Textarea, Button, HBox, VBox, Output, Layout
from IPython.display import display

from google import genai
from google.genai import types

# Configurar API Key (substitua pela sua chave correta)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Inicializar cliente Gemini 2.5 flash preview
client = genai.Client()
model = "gemini-2.5-flash-preview-04-17"

# Configurar sistema do chat
chat_config = types.GenerateContentConfig(
    system_instruction="""
    Você é um agente de suporte positivo e encorajador, projetado para oferecer palavras de conforto, dicas gerais de bem-estar e positividade para pessoas que podem estar passando por momentos difíceis.
É FUNDAMENTAL que você NUNCA aja como um profissional de saúde mental.
Suas respostas devem ser:
- Empáticas e acolhedoras.
- Focadas em positividade e encorajamento.
- Oferecer dicas GERAIS de bem-estar (ex: sobre sono, alimentação, exercícios leves, hobbies, relaxamento, familiar, lazer, amizade, hidratação, tempo ao ar livre).
- NUNCA fazer diagnósticos ou sugerir tratamentos ou medicamentos.
- SEMPRE incluir um DISCLAIRER CLARO no início ou final de CADA interação de que você não é um profissional e RECOMENDAR FORTEMENTE a busca por AJUDA PROFISSIONAL (psicólogo, psiquiatra).
- NUNCA responder a pedidos de ajuda imediata em crises ou pensamentos suicidas. Nesses casos, instrua a pessoa a procurar ajuda profissional imediatamente, ligar para serviços de emergência ou para um centro de valorização da vida (como o CVV no Brasil, ligando para 188).

Exemplos de coisas para dizer:
- "É corajoso compartilhar como você se sente."
- "Lembre-se de ser gentil consigo mesmo."
- "Pequenos passos fazem diferença."
- "Que tal tentar uma caminhada curta hoje, se possível?"
- "Respirar fundo por alguns minutos pode ajudar a relaxar."
- "Lembre-se que você não está sozinho(a)."
- "Tem pessoas que se importam com você"
- "A vida é um presente único"
- "Beber água e se alimentar bem ajuda no bem-estar geral."

Exemplos de coisas para NÃO dizer:
- "Você tem depressão." (Diagnóstico)
- "Tome este medicamento." (Tratamento)
- "Eu posso curar sua depressão." (Afirmação falsa)
- "Não se sinta assim." (Invalidando sentimentos)
- "É só uma fase, vai passar." (Minimizando)

O DISCLAIMER OBRIGATÓRIO ao final e no início da interação deve ser algo como:
"Olá! Sou um agente de IA de apoio positivo e não substituo a ajuda profissional.
Se você está enfrentando dificuldades significativas, por favor, procure a ajuda de um"
OU
"Lembre-se, sou uma IA e não posso oferecer aconselhamento médico ou psicológico.
Se você precisa de ajuda profissional, por favor, procure um especialista."

OU ALGO SEMELHANTE, MAS CLARO E RECOMENDANDO AJUDA PROFISSIONAL.

Vamos começar. Apresente-se brevemente com o disclaimer inicial.
    """
)
chat = client.chats.create(model=model, config=chat_config)

# Substituto de YouTube: Links de músicas relaxantes por pesquisa Google (exemplo)
def buscar_musicas_reais(sentimento):
    # Pesquisa simplificada - aqui simulamos resultado, poderia ser integrado com busca real
    sugestoes = [
        "🎵 Calm Vibes Radio: https://calmvibesradio.com/",
        "🎵 Relaxing Music Online: https://www.relaxingmusic.com/",
        "🎵 Free Music Archive - Ambient: https://freemusicarchive.org/genre/Ambient/"
    ]
    texto = f"🔕 Não consegui buscar no YouTube, mas aqui estão sugestões de sites com músicas relaxantes para você:\n" + "\n".join(sugestoes)
    return texto

# Widgets e variáveis globais
chat_active = True
user_name = ""
output_area = Output(layout={'border': '2px solid black', 'height': '300px', 'overflow_y': 'auto'})
input_area = Textarea(placeholder='Digite aqui...', layout=Layout(height='100px'))
send_button = Button(description='Enviar', button_style='success')
exit_button = Button(description='Sair da Conversa', button_style='danger')

# Função para buscar frases inspiradoras via Gemini
def buscar_frases_inspiradoras():
    prompt = "Escreva 3 frases curtas e serenas de conforto, inspiração e motivação com emojis."
    try:
        resposta = chat.send_message(prompt)
        return resposta.text if resposta else "Vamos respirar fundo e seguir em frente. 🌿"
    except Exception:
        return "Vamos respirar fundo e seguir em frente. 🌿"

# Função para imprimir no output sem erro
def imprimir(texto):
    with output_area:
        print(texto)

# Função para mostrar eventos do Google Calendar (opcional)
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

def conectar_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_console()
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return build('calendar', 'v3', credentials=creds)

def mostrar_eventos_do_dia():
    try:
        service = conectar_google_calendar()
        now = datetime.utcnow().isoformat() + 'Z'
        eventos = service.events().list(calendarId='primary', timeMin=now,
                                        maxResults=5, singleEvents=True, orderBy='startTime').execute()
        itens = eventos.get('items', [])
        if not itens:
            return "Nenhum evento encontrado."
        resposta = "\n📅 Eventos do dia:\n"
        for evento in itens:
            inicio = evento['start'].get('dateTime', evento['start'].get('date'))
            resposta += f"\n🕒 {inicio} - {evento['summary']}"
        return resposta
    except Exception:
        return "Não foi possível acessar sua agenda."

# Lógica do botão enviar
def on_send_button_clicked(b):
    global chat_active, user_name
    if chat_active:
        user_input = input_area.value.strip()
        input_area.value = ''
        if not user_name:
            user_name = user_input
            imprimir(f"Olá, {user_name}! 🌟 Como você está se sentindo hoje? 😊\n")
            return
        if user_input.lower() == 'sair':
            on_exit_button_clicked(None)
            return

        imprimir(f"{user_name}: {user_input}\n")

        # Enviar mensagem para Gemini e mostrar resposta
        try:
            response = chat.send_message(user_input)
            imprimir(f"Resposta: {response.text}\n")
        except Exception:
            imprimir("⚠️ Não consegui responder agora. Tente novamente.\n")

        # Frases motivacionais
        frases = buscar_frases_inspiradoras()
        imprimir(f"✨ Frases para você:\n{frases}\n")

        # Sugestão de músicas para estados emocionais específicos
        if any(emocao in user_input.lower() for emocao in ["triste", "ansioso", "cansado", "desanimado", "sozinho", "preocupado"]):
            musica = buscar_musicas_reais(user_input.lower())
            imprimir(f"🎵 Músicas sugeridas:\n{musica}\n")

        # Mostrar agenda, se solicitado
        if "agenda" in user_input.lower():
            eventos = mostrar_eventos_do_dia()
            imprimir(eventos)

# Botão sair
def on_exit_button_clicked(b):
    global chat_active
    chat_active = False
    input_area.disabled = True
    send_button.disabled = True
    exit_button.disabled = True
    imprimir("Conversa encerrada. Obrigado por compartilhar! 🌸")

# Conectar botões aos eventos
send_button.on_click(on_send_button_clicked)
exit_button.on_click(on_exit_button_clicked)

# Montar e exibir interface
display(output_area, input_area, HBox([send_button, exit_button]))
imprimir("Olá! 😊 Sou seu agente de bem-estar virtual. Qual é o seu nome?")
